# 1. Load the labeled and unlabeled comments.

In [1]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
# !pip install transformers==4.3.2
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from typing import Union
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece

##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

#client = bigquery.Client()

In [2]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [3]:
# Load the filtered unlabeled comments and all the labeled comments.
fb_comments_unlabeled_df = pd.read_pickle("./unlabeled_filtered_comments.pkl")
fb_comments_labeled_df = pd.read_pickle("./labeled_comments.pkl")

print("You have ", len(fb_comments_unlabeled_df), " unlabeled comments")
print("You're using: ", len(fb_comments_labeled_df), " labeled comments.")
n_neg, n_pos = fb_comments_labeled_df["sentiment_label"].value_counts()[0], fb_comments_labeled_df["sentiment_label"].value_counts()[1]
print("Negative: %d%% (%d)" %(n_neg*100/len(fb_comments_labeled_df), n_neg))
print("Positive: %d%% (%d)" %(n_pos*100/len(fb_comments_labeled_df), n_pos))

You have  6794  unlabeled comments
You're using:  959  labeled comments.
Negative: 74% (710)
Positive: 25% (249)


# 2. Get train and test sets. 

In [4]:
def get_train_test(labeled_comments: pd.core.frame.DataFrame, test_size: Union[int, float],
                   new_split: bool = False, files: dict = {}) -> tuple:
    if new_split:
        if isinstance(test_size, int):
            test_size = test_size / labeled_comments.shape[0]
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state = 0)
        sss.get_n_splits(labeled_comments["message"].values, labeled_comments["sentiment_label"].values)
        train_labeled_data = None
        test_labeled_data = None
        for train_index, test_index in sss.split(labeled_comments["message"].values, labeled_comments["sentiment_label"].values):
            train_labeled_data = labeled_comments.iloc[train_index]
            test_labeled_data = labeled_comments.iloc[test_index]
        train_labeled_data.to_pickle("./train_labeled_data.pkl")
        test_labeled_data.to_pickle("./test_labeled_data.pkl")
    else:
        train_labeled_data = pd.read_pickle(files["train"])
        test_labeled_data = pd.read_pickle(files["test"])
    
    return train_labeled_data, test_labeled_data
        

             
def prepare_data(train_labeled_comments: pd.core.frame.DataFrame, test_labeled_comments: pd.core.frame.DataFrame,
                 unlabeled_comments: pd.core.frame.DataFrame, percentage: Union[int, float],
                display_class_percentages: bool = False) -> tuple:
    
    # Grab the requested percentage (with respect to unlabeled data) of labeled comments for training. 
    if isinstance(percentage, int):
        percentage = percentage/100
    if percentage != .1:
        labeled_comments = train_labeled_comments.sample(frac=1).reset_index(drop=True) # shuffle
        n_labeled = train_labeled_comments.shape[0]
        n_unlabeled = unlabeled_comments.shape[0]
        n_wanted = round(percentage * n_unlabeled)
        if n_wanted > n_labeled:
            raise Exception("ERROR. Insufficient amount of labeled comments, try a lower percentage.")
        n_drop = n_labeled - n_wanted
        train_labeled_comments.drop(train_labeled_comments.tail(n_drop).index, inplace=True)
        print("You're using: ", train_labeled_comments.shape[0], " labeled comments.\n")
    
    # Prepare the data for the DataLoader function below.
    # 1. Make tuples with labeled data: (feature, label). 
    unlabeled_arr = np.array([(message, "UNK_UNK") for message in unlabeled_comments["message"].values])
    train_arr = np.array([(row["message"], row["sentiment_label"]) for _, row in train_labeled_comments.iterrows()])
    test_arr = np.array([(row["message"], row["sentiment_label"]) for _, row in test_labeled_comments.iterrows()])
    
    if display_class_percentages: 
        # Use code of previous versions (ugly).
        train_labeled_data = train_arr
        d_train = {"pos": 0, "neg": 0, "UNK_UNK": 0}
        d_test = d_train.copy()
        for _, label in train_labeled_data:
            d_train[label] += 1
        print("Percentages in train:")
        print("Negative: %d%% (%d)" %(round(d_train["neg"]*100/len(train_labeled_data)), d_train["neg"]))
        print("Positive:  %d%% (%d)" %(round(d_train["pos"]*100/len(train_labeled_data)), d_train["pos"]))
        print("")

        test_labeled_data = test_arr
        for _, label in test_labeled_data:
            d_test[label] += 1
        print("Percentages in test: ")
        print("Negative: %d%%" %(round(d_test["neg"]*100/len(test_labeled_data))))
        print("Positive:  %d%%" %(round(d_test["pos"]*100/len(test_labeled_data))))
        
    # 2. Create mask arrays.
    unlabeled_masks = np.zeros(unlabeled_arr.shape[0], dtype=bool)
    train_masks = np.ones(train_arr.shape[0], dtype=bool)
    test_masks = np.ones(test_arr.shape[0], dtype=bool)
    # 3. Extend the train data with the unlabeled data.
    train_arr = np.concatenate((train_arr, unlabeled_arr), axis=0)
    train_masks = np.concatenate((train_masks, unlabeled_masks))
    
    
    return train_arr, train_masks, test_arr, test_masks

new_split = False
test_size = 280
files = {"train": "train_labeled_data.pkl", "test": "test_labeled_data.pkl"}
train_labeled_data, test_labeled_data = get_train_test(fb_comments_labeled_df, test_size, new_split=new_split, files=files)
labeled_data_percentage = .07 # Used 1, 3, 5, 10 % for linear lr experiment
display = True
train_examples, train_label_masks, test_examples, test_label_masks = prepare_data(train_labeled_data, 
                                                                                  test_labeled_data,
                                                                                  fb_comments_unlabeled_df,
                                                                                  labeled_data_percentage,
                                                                                 display_class_percentages=display)

#print("Now your're using: ", len(train_examples), " labeled comments.")

You're using:  476  labeled comments.

Percentages in train:
Negative: 73% (346)
Positive:  27% (130)

Percentages in test: 
Negative: 74%
Positive:  26%


In [5]:
train_labeled_data.shape

(476, 5)

# 3.1 Architecture parameters.

In [6]:
#------------------------------
#   The Generator as in 
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

In [7]:
label_list = ["neg", "pos", "UNK_UNK"]
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 32

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator, 
# each of the size of the output space
num_hidden_layers_g = 1; 
# number of hidden layers in the discriminator, 
# each of the size of the input space
num_hidden_layers_d = 1; 
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2

# Replicate labeled data to balance poorly represented datasets, 
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 5
multi_gpu = True
# Scheduler
apply_scheduler = True # CHANGED THIS
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "KB/bert-base-swedish-cased"


In [8]:
local_files_only = True
transformer = AutoModel.from_pretrained(model_name, local_files_only=local_files_only)
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=local_files_only)

In [9]:
import os
os.getcwd()

'/home/jupyter/src'

In [10]:
# The config file is required to get the dimension of the vector produced by 
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():    
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)

# 3.2 DataLoader function.

In [11]:
def generate_data_loader(input_examples, label_masks, label_map, batch_size, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples
  label_mask_rate = sum(label_masks)/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples): 
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))
  
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization 
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)
  
  print(len(examples))
  print(len(text[0]))
  print(len(encoded_sent))
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
            dataset,  # The training samples.
            sampler = sampler(dataset),
            drop_last = True,
            batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


# 4. Training procedure.

In [12]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i

# 4. Training procedure.

In [13]:
#--------------------------------
#  Helper fucntions for the training procedure
#--------------------------------
from sklearn.metrics import (PrecisionRecallDisplay, precision_recall_fscore_support, precision_recall_curve)

# Precision = TP/(TP+FP) Ratio between true positives and all positived
# Recall = TP/(TP+FN) How well does the model identify true positives
def calculate_precision_recall(all_preds, all_label_ids):
    scores = precision_recall_fscore_support(all_preds, all_label_ids, average=None)
    p_r_f_for_class = dict()
    for i in range(2):
        label = 'Positive' if i != 0 else "Negative"
        p_r_f_for_class[label] = (scores[0][i], scores[1][i], scores[2][i])

    print('precision / recall / f-score for class: ', p_r_f_for_class)
    precision_arr, recall_arr, _ = precision_recall_curve(all_label_ids, all_preds)
    
    return p_r_f_for_class

In [14]:
data = []
save_data = True
percentage = str(labeled_data_percentage*100) if isinstance(labeled_data_percentage, float) else str(labeled_data_percentage)
st = time.time()
for i in range(3):
    print("\nRUN NUMBER: ", i+1)
    
    train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, 
                                            batch_size, do_shuffle = True, balance_label_examples = apply_balance)

    test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, 
                                       batch_size, do_shuffle = False, balance_label_examples = False)
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    #models parameters
    transformer_vars = [i for i in transformer.parameters()]
    d_vars = transformer_vars + [v for v in discriminator.parameters()]
    g_vars = [v for v in generator.parameters()]

    #optimizer
    dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
    gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 

    #scheduler
    if apply_scheduler:
      num_train_examples = len(train_examples)
      num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
      num_warmup_steps = int(num_train_steps * warmup_proportion)
    
      # USE LINEAR LEARNING RATE IN THIS EXPERIMENT!
      scheduler_d = get_linear_schedule_with_warmup(dis_optimizer, 
                                               num_warmup_steps = num_warmup_steps, num_training_steps=num_train_steps)
      scheduler_g = get_linear_schedule_with_warmup(gen_optimizer, 
                                               num_warmup_steps = num_warmup_steps, num_training_steps=num_train_steps)

    # For each epoch...
    for epoch_i in range(0, num_train_epochs):
        # ========================================
        #               Training
        # ========================================
        # Perform one full pass over the training set.
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, num_train_epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        tr_g_loss = 0
        tr_d_loss = 0

        # Put the model into training mode.
        transformer.train() 
        generator.train()
        discriminator.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every print_each_n_step batches.
            if step % print_each_n_step == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            b_label_mask = batch[3].to(device)

            # Encode real data in the Transformer
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs[-1]

            # Generate fake data that should have the same distribution of the ones
            # encoded by the transformer. 
            # First noisy input are used in input to the Generator
            noise = torch.zeros(b_input_ids.shape[0],noise_size, device=device).uniform_(0, 1)
            # Gnerate Fake data
            gen_rep = generator(noise)

            # Generate the output of the Discriminator for real and fake data.
            # First, we put together the output of the tranformer and the generator
            disciminator_input = torch.cat([hidden_states, gen_rep], dim=0)
            # Then, we select the output of the disciminator
            features, logits, probs = discriminator(disciminator_input)

            # Finally, we separate the discriminator's output for the real and fake
            # data
            features_list = torch.split(features, batch_size)
            D_real_features = features_list[0]
            D_fake_features = features_list[1]

            logits_list = torch.split(logits, batch_size)
            D_real_logits = logits_list[0]
            D_fake_logits = logits_list[1]

            probs_list = torch.split(probs, batch_size)
            D_real_probs = probs_list[0]
            D_fake_probs = probs_list[1]

            #---------------------------------
            #  LOSS evaluation
            #---------------------------------
            # Generator's LOSS estimation
            g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
            g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
            g_loss = g_loss_d + g_feat_reg

            # Disciminator's LOSS estimation
            logits = D_real_logits[:,0:-1]
            log_probs = F.log_softmax(logits, dim=-1)
            # The discriminator provides an output for labeled and unlabeled real data
            # so the loss evaluated for unlabeled data is ignored (masked)
            label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
            per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
            per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
            labeled_example_count = per_example_loss.type(torch.float32).numel()

            # It may be the case that a batch does not contain labeled examples, 
            # so the "supervised loss" in this case is not evaluated
            if labeled_example_count == 0:
              D_L_Supervised = 0
            else:
              D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)

            D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
            D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
            d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

            #---------------------------------
            #  OPTIMIZATION
            #---------------------------------
            # Avoid gradient accumulation
            gen_optimizer.zero_grad()
            dis_optimizer.zero_grad()

            # Calculate weigth updates
            # retain_graph=True is required since the underlying graph will be deleted after backward
            g_loss.backward(retain_graph=True)
            d_loss.backward() 

            # Apply modifications
            gen_optimizer.step()
            dis_optimizer.step()

            # A detail log of the individual losses
            #print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}".
            #      format(D_L_Supervised, D_L_unsupervised1U, D_L_unsupervised2U,
            #             g_loss_d, g_feat_reg))

            # Save the losses to print them later
            tr_g_loss += g_loss.item()
            tr_d_loss += d_loss.item()

            # Update the learning rate with the scheduler
            if apply_scheduler:
              scheduler_d.step()
              scheduler_g.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss_g = tr_g_loss / len(train_dataloader)
        avg_train_loss_d = tr_d_loss / len(train_dataloader)             

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
        print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #     TEST ON THE EVALUATION DATASET
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our test set.
        print("")
        print("Running Test...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        transformer.eval() #maybe redundant
        discriminator.eval()
        generator.eval()

        # Tracking variables 
        total_test_accuracy = 0

        total_test_loss = 0
        nb_test_steps = 0

        all_preds = []
        all_labels_ids = []

        #loss
        nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

        # Evaluate data for one epoch
        for batch in test_dataloader:

            # Unpack this training batch from our dataloader. 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        
                model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
                hidden_states = model_outputs[-1]
                _, logits, probs = discriminator(hidden_states)
                ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
                filtered_logits = logits[:,0:-1]
                # Accumulate the test loss.
                total_test_loss += nll_loss(filtered_logits, b_labels)

            # Accumulate the predictions and the input labels
            _, preds = torch.max(filtered_logits, 1)
            all_preds += preds.detach().cpu()
            all_labels_ids += b_labels.detach().cpu()

        # Report the final accuracy for this validation run.
        all_preds = torch.stack(all_preds).numpy()
        print(len(all_preds))
        all_labels_ids = torch.stack(all_labels_ids).numpy()
        test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
        p_r_f_for_class = calculate_precision_recall(all_preds, all_labels_ids)
        print("  Accuracy: {0:.3f}".format(test_accuracy))

        # Calculate the average loss over all of the batches.
        avg_test_loss = total_test_loss / len(test_dataloader)
        avg_test_loss = avg_test_loss.item()

        # Measure how long the validation run took.
        test_time = format_time(time.time() - t0)

        print("  Test Loss: {0:.3f}".format(avg_test_loss))
        print("  Test took: {:}".format(test_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss generator': avg_train_loss_g,
                'Training Loss discriminator': avg_train_loss_d,
                'Valid. Loss': avg_test_loss,
                'Valid. Accur.': test_accuracy,
                'Training Time': training_time,
                'Test Time': test_time,
                'PRF': p_r_f_for_class
            }
        )
    data.append(training_stats)
    
if save_data:
    print("Data saved! Took me %.2f seconds." %(time.time()-st))
    np.save("linear_lr_results_" + percentage + "_percent", 
            data)




RUN NUMBER:  1
8222
66
64
280
87
64

======== Epoch 1 / 5 ========
Training...
  Batch    10  of    256.    Elapsed: 0:00:11.
  Batch    20  of    256.    Elapsed: 0:00:22.
  Batch    30  of    256.    Elapsed: 0:00:32.
  Batch    40  of    256.    Elapsed: 0:00:43.
  Batch    50  of    256.    Elapsed: 0:00:53.
  Batch    60  of    256.    Elapsed: 0:01:04.
  Batch    70  of    256.    Elapsed: 0:01:14.
  Batch    80  of    256.    Elapsed: 0:01:25.
  Batch    90  of    256.    Elapsed: 0:01:36.
  Batch   100  of    256.    Elapsed: 0:01:46.
  Batch   110  of    256.    Elapsed: 0:01:57.
  Batch   120  of    256.    Elapsed: 0:02:07.
  Batch   130  of    256.    Elapsed: 0:02:18.
  Batch   140  of    256.    Elapsed: 0:02:28.
  Batch   150  of    256.    Elapsed: 0:02:39.
  Batch   160  of    256.    Elapsed: 0:02:49.
  Batch   170  of    256.    Elapsed: 0:03:00.
  Batch   180  of    256.    Elapsed: 0:03:11.
  Batch   190  of    256.    Elapsed: 0:03:21.
  Batch   200  of    256.  

In [ ]:
np.save("results_linear_lr_10_percent", data)